<a href="https://colab.research.google.com/github/Marlucben/Challenge-ONE-Data-Science-Telecom-X-parte-2/blob/main/Challenge_parte_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Carga de base de datos ##

In [3]:
import pandas as pd
import numpy as np
url = 'https://raw.githubusercontent.com/Marlucben/Challenge-ONE-Data-Science-Telecom-X-parte-2/refs/heads/main/df_nuevo.json'
df = pd.read_json(url , orient="records", lines=True)
df.head()

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total,Charges.Days
0,0002-ORFBO,NO,FEMALE,0,YES,YES,9,YES,NO,ONE YEAR,YES,MAILED CHECK,65.6,593.30,2.2
1,0003-MKNFE,NO,MALE,0,NO,NO,9,YES,YES,MONTH-TO-MONTH,NO,MAILED CHECK,59.9,542.40,2.0
2,0004-TLHLJ,YES,MALE,0,NO,NO,4,YES,NO,MONTH-TO-MONTH,YES,ELECTRONIC CHECK,73.9,280.85,2.5
3,0011-IGKFF,YES,MALE,1,YES,NO,13,YES,NO,MONTH-TO-MONTH,YES,ELECTRONIC CHECK,98.0,1237.85,3.3
4,0013-EXCHZ,YES,FEMALE,1,YES,NO,3,YES,NO,MONTH-TO-MONTH,YES,MAILED CHECK,83.9,267.40,2.8
